In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [2]:
import numpy as np
import pandas as pd
import time
from typing import Any
import json

reviews_path = "/Users/alef/Documents/hungry-ai/data/yelp_dataset/yelp_academic_dataset_review.json"
image_tags_path = "/Users/alef/Documents/hungry-ai/data/real/image_tags.json"

reviews_data: dict[str, list[Any]] = {
    "user_id": [],
    "rating": [],
    "business_id": [],
}

with open(reviews_path) as f:
    for line_raw in f:
        line = json.loads(line_raw)
        reviews_data["user_id"].append(line.get("user_id", None))
        reviews_data["business_id"].append(line.get("business_id", None))
        reviews_data["rating"].append(line.get("stars", None))
        
reviews_df = pd.DataFrame(reviews_data)
print(reviews_df.head())

                  user_id  rating             business_id
0  mh_-eMZ6K5RLWhZyISBhwA     3.0  XQfwVwDr-v0ZS3_CbbE5Xw
1  OyoGAe7OKpv6SyGZT5g77Q     5.0  7ATYjTIgM3jUlt4UM3IypQ
2  8g_iMtfSiwikVnbP2etR0A     3.0  YjUWPpI6HXG530lwP-fb2A
3  _7bHUi9Uuf5__HHc_Q8guQ     5.0  kxX2SOes4o-D3ZQBkiMRfA
4  bcjbaE6dDog4jkNY91ncLQ     4.0  e4Vwtrqf-wpJfwesgvdgxQ


In [47]:
image_tags_path = "/Users/alef/Documents/hungry-ai/data/real/image_tags.json"

tags = []
images = []
businesses = []
scores = []

with open(image_tags_path, encoding = 'latin-1') as f:
    for line_raw in f:
        line = json.loads(line_raw, strict = False)
        for el in line["labels"]:
            tags.append(el["description"])
            images.append(line["photo_id"])
            businesses.append(line["business_id"])
            scores.append(el["score"])
            
df = pd.DataFrame(list(zip(tags,images,businesses,scores)), columns = ["tags","images","businesses","scores"])
df = df.groupby(["tags",'images','businesses'])['scores'].sum().reset_index()

image_indices = pd.DataFrame(df['images'].unique()).reset_index().rename(columns = {'index' : 'image_index', 0 : 'images'})
df = df.merge(image_indices, on = 'images')

business_indices = pd.DataFrame(df['businesses'].unique()).reset_index().rename(columns = {'index' : 'business_index', 0 : 'businesses'})
df = df.merge(business_indices, on = 'businesses')

tag_indices = pd.DataFrame(df['tags'].unique()).reset_index().rename(columns = {'index' : 'tag_index', 0 : 'tags'})
df = df.merge(tag_indices, on = 'tags')

m = df['image_index'].max()+1
k = df['tag_index'].max()+1

I = np.zeros((m,k+1))

for i,row in df.iterrows():
    I[row['image_index']][row['tag_index']] = row['scores']

for row in range(m):
    row_sum = sum(I[row])
    if(row_sum != 0):
        for col in range(k):
            I[row][col] /= row_sum

for row in range(m):
    I[row][k] = 1

print(I)

[[0.10923699 0.09649112 0.09391821 ... 0.         0.         1.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]
 ...
 [0.         0.         0.         ... 0.         0.         1.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [43]:
image_indices.max()

image_index                     31586
images         zzcYA1JoRoJGwVk5pTjSaQ
dtype: object

In [48]:
BI = df[["businesses","images", "image_index"]].drop_duplicates()

reviews = BI.merge(reviews_df,how = "inner", left_on = "businesses", right_on = "business_id")[["user_id","images","rating","image_index"]]

reviews

,user_id,images,rating,image_index
0,yWmupwLUZ2Sa-u8bvs853Q,-CkHJ95FbPjQKtxJmplNsA,4.0,0
1,vffKQc_WQMYFGY4JS5VAOw,-CkHJ95FbPjQKtxJmplNsA,3.0,0
2,qj_DyM6laBseHq-ylAmsTw,-CkHJ95FbPjQKtxJmplNsA,2.0,0
3,oS2U_MtAcpjoF9GExypZ8A,-CkHJ95FbPjQKtxJmplNsA,1.0,0
4,cMAiTrM0Fxm8vKLQag53jg,-CkHJ95FbPjQKtxJmplNsA,2.0,0
...,...,...,...,...
19550190,FZbx7q0Lp_XIMSHrMQ1m3w,tXPUgN0zmYZC8IOePsAZQw,1.0,20035
19550191,DMPzVexqwVTweZ-rZst6Zw,tXPUgN0zmYZC8IOePsAZQw,4.0,20035
19550192,EYXx17hVkCkW84xvTBncQQ,tXPUgN0zmYZC8IOePsAZQw,1.0,20035
19550193,e8Lfi7Km12QbcQFGWzuo1w,tXPUgN0zmYZC8IOePsAZQw,4.0,20035


In [49]:
user_index = reviews["user_id"].drop_duplicates().reset_index().rename(columns = {'index' : 'user_index'})
reviews = reviews.merge(user_index, on = 'user_id')

In [50]:
from sklearn.model_selection import train_test_split  # type: ignore[import]

reviews_train, reviews_test = train_test_split(
    reviews, train_size=0.8, random_state=42069
)

reviews_train[["user_index", "image_index", "rating"]].sort_values("user_index").to_csv("../data/real/reviews_train_cpp.csv", index=None, header=False, sep=" ")

In [51]:
with open("../data/real/I.txt", "w") as f:
    for i in I:
        f.write(" ".join([str(j) for j in i]) + "\n")

In [52]:
print(len(reviews_train),len(reviews['user_index'].drop_duplicates()), m, k)

15640156 324749 31587 2635
